In [ ]:
##### Script Version
##### 19 April 2024 Commits
##### Bug Out 1

In [2]:
"""IMPORTS"""

from arcgis.gis import GIS
from arcgis.gis import Item
from arcgis import __version__
from arcgis.features import FeatureLayerCollection, Table
from arcgis.apps.expbuilder import WebExperience
from arcgis.mapping import WebMap
from datetime import datetime

import pandas as pd
import tempfile

import uuid
import json
import tempfile
import os
import shutil

from getpass import getpass

# CONFIGURE TRANSFER HERE
##Run this cell each time you update the item id

In [ ]:
"""CONSTANTS""" 

ORIGIN_TRANSFER_USER = "" #Examples - ['truck_transfer', 'auto_transfer', 'manu_transfer','retail_transfer', 'realestate_transfer', 'realestate2_transfer']
ITEM_ID = "" # INSERT HERE

#Set to True if you want to have content moved to a different user in the destination
REASSIGN = False
#If REASSIGN is set to True then MOVE_TO has to be set to the username you want the items to go to in the destination org
MOVE_TO = '' #Example - john7126@esri.com_esri_cgs

# Run only at start up 
### TRANSFER CODE (Run this the first time you open the notebook, but you don't need to run this section each time -- once you are logged in you don't need to re-run this each time)

In [ ]:
print("Connecting via named user")
origin_pass = getpass(prompt=f"Enter the password for user {ORIGIN_TRANSFER_USER}: ")
origin = GIS('https://www.arcgis.com', ORIGIN_TRANSFER_USER, origin_pass, expiration=9999)
print("Connection Successful.")
print("Logged into portal as: " + origin.properties.user.username)

#Log into Destination 
o_url = origin.url
print(f"Attempting loging at AGOL with url {o_url}")

###Log in via named user method
if any(keyword in o_url for keyword in ['manucomm', 'truckcomm', 'commteamretail', 'automotive', 'commre', 'commteamre']):
    DESTINATION_TRANSFER_USER = "cgs_transfer"
    print(f"Logging in with {DESTINATION_TRANSFER_USER}")
    #destination_pass = getpass(prompt=f"Enter the password for user {DESTINATION_TRANSFER_USER}: ")
    destination_pass = origin_pass
    DESTINATION_URL = "https://esri-cgs.maps.arcgis.com/"
    CATALOG_ID = "9f82678130ef47aeb942bf87ab99a3eb"
    destination = GIS(DESTINATION_URL, DESTINATION_TRANSFER_USER, destination_pass, expiration=9999)
    print("Successfully logged in as: " + destination.properties.user.username)
elif any(keyword in o_url for keyword in ['banking', 'inscomm']):
    DESTINATION_TRANSFER_USER = "fsi_transfer"
    print(f"Logging in with {DESTINATION_TRANSFER_USER}:")
    #destination_pass = getpass(prompt=f"Enter the password for user {DESTINATION_TRANSFER_USER}: ")
    destination_pass = origin_pass
    DESTINATION_URL = "https://fsi.maps.arcgis.com/"
    CATALOG_ID = "98c72904ee4640a5b3707ee6f2dd1246"
    destination = GIS(DESTINATION_URL, DESTINATION_TRANSFER_USER, destination_pass, expiration=9999)
    print("Successfully logged in as: " + destination.properties.user.username)
else:
    DESTINATION_URL = input("Enter the portal URL for the destination portal: ")
    DESTINATION_TRANSFER_USER = input("Enter the username for the destination portal: ")
    destination_pass = getpass(prompt=f"Enter the password for user {DESTINATION_TRANSFER_USER}: ")
    CATALOG_ID = input("Enter the catalog itemid from the destination portal: ")
    destination = GIS(DESTINATION_URL, DESTINATION_TRANSFER_USER, destination_pass, expiration=9999)
    print("Successfully logged in as: " + destination.properties.user.username)

### Helper Functions

In [ ]:
ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene', 'Dashboard',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])

FILE_BASED_ITEM_TYPES = frozenset(['File Geodatabase','CSV', 'Image', 'KML', 'Locator Package',
                                  'Map Document', 'Shapefile', 'Microsoft Word', 'PDF',
                                  'Microsoft Powerpoint', 'Microsoft Excel', 'Layer Package',
                                  'Mobile Map Package', 'Geoprocessing Package', 'Scene Package',
                                  'Tile Package', 'Vector Tile Package'])

RELATIONSHIP_TYPES = frozenset(['Map2Service', 'WMA2Code',
                                'Map2FeatureCollection', 'MobileApp2Code', 'Service2Data',
                                'Service2Service'])

_version = [int(i) for i in __version__.split('.')]

#Set to False to keep items in the Origin Org in place. True is the default.
MOVE_ITEMS = True

#Create or get folder in destination user content
def get_or_create_folder(gis: GIS, folder_name: str, user: str) -> str:
    # Check if the folder exists
    folders = gis.users.get(user).folders
    existing_folder = next((f for f in folders if f['title'] == folder_name), None)

    if existing_folder:
        print(f"Folder '{folder_name}' already exists.")
        return folder_name
    else:
        # Create the folder
        new_folder = gis.content.create_folder(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
        return folder_name

#Get the specified item as an object and then get the title
item_origin = origin.content.get(ITEM_ID)
fldr_name = get_or_create_folder(destination, item_origin.title, destination.properties.user.username)

def export_resources(item, save_path=None, file_name=None):
    """
    Helper function, from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    Export's the data's resources as a zip file
    """
    
    url = f'{item._gis._portal.resturl}content/users/{item._user_id}/items/{item.itemid}/resources/export'
    if save_path is None:
        save_path = tempfile.gettempdir()
    if file_name is None:
        file_name = f"{uuid.uuid4().hex[:6]}.zip"
    params = {'f' : 'zip'}
    con = item._gis._portal.con
    resources = con.get(url, params=params,
                        out_folder=save_path,
                        file_name=file_name,
                        try_json=False)
    return resources

def get_layer_item_ids(wm) -> list:
    """
    Helper function from https://developers.arcgis.com/python/guide/cloning-content/
    
    Returns the related items in a webmap.
    
    Params:
        wm (argis.gis.Item): Webmap item to be inspected.
    Returns:
        wm_id_list (list): List of related items in the web map.
    """
    wmo = WebMap(wm)
    wm_id_list = []
    
    for layer in wmo.layers:
        try:
            fsvc = FeatureLayerCollection(layer['url'][:-1], origin)
            if not fsvc.properties['serviceItemId'] in wm_id_list:
                wm_id_list.append(fsvc.properties['serviceItemId'])
        except Exception as e:
            continue
    return wm_id_list

def flatten_json(y):
    out = {}
 
    def flatten(x, name=''):
 
        # If the Nested key-value
        # pair is of dict type
        if type(x) is dict:
 
            for a in x:
                flatten(x[a], name + a + '_')
 
        # If the Nested key-value
        # pair is of list type
        elif type(x) is list:
 
            i = 0
 
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
 
    flatten(y)
    return out

def get_dash_wm(dash) -> list:
    """
    From https://developers.arcgis.com/python/guide/cloning-content/#helper-functions
    Returns a list of all Web Maps participating in a Dashboard. 
    Arguments:
        dash (item): Dashboard to return participating Web Maps from.
    Returns:
        (list): All Web Maps partipating in the dashboard. 
    """
    
    dash_json = dash.get_data()
    flattened_dash = flatten_json(dash_json)
    
    webmap_list = []
    
    for k, v in flattened_dash.items():
        if 'itemId' in k:
            item = origin.content.get(v)
            if item.type == 'Web Map' and item not in webmap_list:
                webmap_list.append(item)
    
    print(webmap_list)
    return webmap_list

### Transfer Functions

In [ ]:
def wc_transfer(destination: GIS, 
                records: Table = None, 
                items: list = [], 
                logging: bool = False) -> list:
    
    """
    Performs a web content transfer of items from an origin to destination AGOL. 
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the given items
        items (list): a list of Items to be transferred.
        records (Table): Hosted Table item to memoize transfers to. Must be passed if logging=True
        logging (bool): If True, enables catalog memoization, pushing a transfer record to a Hosted Table in AGOL.  
    """
    
    origin_to_destination_ids = {}
    
    for item in items:
        if item.owner != ORIGIN_TRANSFER_USER and MOVE_ITEMS:
            item.reassign_to(ORIGIN_TRANSFER_USER)
    
    item_titles = [item.title for item in items]
    
    print("Performing Web Content transfer for the following items: ")
    for title in item_titles:
        print(f'>> {title}')
        
    for item in items:
        try:
            if item.groupDesignations == 'livingatlas' or 'livingatlas' in item.groupDesignations:
                print(f"{item.title} is a Living Atlas item and therefore can only be referenced, not copied. Removing it from transfer.")
                return
            if 'Requires Subscription' in item.typeKeywords:
                print(f"{item.title} is a premium subscription item and therefore can only be referenced, not copied. Removing it from transfer.")
                return
            if 'utility.arcgis.com/usrsvcs' in item.url:
                print(f"{item.title} is a referenced  item and therefore can only be referenced, not copied. Removing it from transfer.")
                return
        except TypeError:
            continue
                                
    destination_items = destination.content.clone_items(items, folder=fldr_name) ##Changed Folder
    
    print("Web Content transfer: Item(s) cloned successfully. Updating tags ... ")
    now = datetime.now()
    tag = f"src_{origin.properties['urlKey']}_{now.month}/{now.day}/{now.year}-{now.hour}:{now.minute}"
    
    for item in destination_items:
        item.update({'tags': tag})
    for item, destitem in zip(items, destination_items):
        destitem.update({'tags': item.tags})
        
    # build origin to destination map and memoize to catalog
    origin_item_index = 0
    for destination_item in destination_items:            
        origin_to_destination_ids[item.id] = destination_item.id
        
        if item.id == ITEM_ID:
            try:
                destination_item.move(fldr_name) ##Changed Folder
            except Exception:
                continue
                
        if logging:   
            
            try:
                adds = {"attributes":
                    {
                        "source_id": items[origin_item_index].id,
                        "destination_id": destination_item.id,
                        "title": destination_item.title,
                        "owner": destination_item.owner,
                        "transfer_date": str(datetime.now())
                    }
                }
            except IndexError:
                print(f"failed to log {destination_item.title}. Item was still brought over to destination.")
            
            records.edit_features(adds=[adds])
        
        origin_item_index += 1
        
    print("Web Content Transfer: complete.")
    
    for item in destination_items:
        try:
            children = get_layer_item_ids(wm=item)
            for child in children:
                child_item = destination.content.get(child)
                child_item.move(fldr_name) ##Changed Folder
        except Exception as e:
            continue
        
    me = destination.properties.user
    items_in_folder = me.items(folder=fldr_name)
    folder_item_ids = {item['title']: item['id'] for item in items_in_folder}
    folder_item_urls = {item['title']: item['url'] for item in items_in_folder}
    
    
    # Swizzling
    for item in destination_items:
        if item.type == "Web Map":
            wm = WebMap(item)
            for layer in wm.layers:
                if "layers" in layer and layer["title"] == "Group Layer":
                    for sub_layer in layer["layers"]:
                        sub_layer_title = sub_layer['title']
                        sub_layer['url'] = folder_item_urls[sub_layer_title]
                        sub_layer['itemId'] = folder_item_ids[sub_layer_title]
            wm.update()

    return destination_items
    
def dash_transfer(destination: GIS, 
                  dash: Item, 
                  swizzle: bool = True, 
                  records: Table = None, 
                  logging: bool = False) -> None:
    
    """    
    Performs a web content transfer of a dashboard to destination AGOL.
    
    Arguments:
        destination (GIS): Destination GIS for the given Dashboard
        dash (Item): A Dashboard item in the origin GIS
        swizzle (bool): If True, enables JSON swizzling to map keys to values. Future proofing for ArcGIS API 2.2 release.
        records (Table): Hosted Table item to memoize transfers to. Must be passed if logging=True
        logging (bool): If True, enables catalog memoization, pushing a transfer record to a Hosted Table in AGOL.  
    """
        
    dash_elements = get_dash_wm(dash=dash)
    wm_items = {} # origin to destination ids
    
    if dash.owner != ORIGIN_TRANSFER_USER and MOVE_ITEMS:
        dash.reassign_to(ORIGIN_TRANSFER_USER)
    
    try:     
        if dash.groupDesignations == 'livingatlas':
            print(f"{item.title} is a Living Atlas item and therefore can only be referenced, not copied. Aborting this dash transfer.")
            return
        if 'Requires Subscription' in dash.typeKeywords:
            print(f"{item.title} is a premium subscription item and therefore can only be referenced, not copied. Aborting this dash transfer.")
            return
    except TypeError:
        # We pass here because a TypeError indicates that there are no issues with typeKeywords or none exist.
        pass
            
    print(f"Creating destination folder for Dashboard {dash.title} ...")
    if dash.id == ITEM_ID:
        destination.content.create_folder(fldr_name) ##Changed Folder
    
    
    dash_elements = list(set(dash_elements))
    for ele in dash_elements:
        
        ele_itm = origin.content.get(ele)
        if ele_itm.owner != ORIGIN_TRANSFER_USER and MOVE_ITEMS:
            ele_itm.reassign_to(ORIGIN_TRANSFER_USER)
                    
        # if the item participating in the dashboard has not yet been cloned: 
        try:
            print(f"Transferring {ele_itm.title} to destination org, moving to Web Content transfer workflow ... ")
            wc = wc_transfer(destination=destination, items=[ele_itm])
                       
        except IndexError:
            print(f"Item {ele.title} has already been transferred, applying destination-side edits ... ")
            ele_from_search = destination.content.search(query=f"typekeywords:source-{ele.id}")[0]
            wm_items[ele.id] = ele_from_search.id
            ele_from_search.move(fldr_name) ##Changed Folder

    print(f"Participating items handled, transferring dashboard {dash.title} ... ")

    if swizzle == False:
        dest_dash = destination.content.clone_items(items=[dash], item_mapping=wm_items, folder=fldr_name) ##Changed Folder
    else:
        dest_dash = destination.content.clone_items(items=[dash], folder=fldr_name) ##Changed Folder

    
    now = datetime.now()
    tag = f"src_{origin.properties['urlKey']}_{now.month}/{now.day}/{now.year}-{now.hour}:{now.minute}"

    for item in dest_dash:
        item.update({'tags': tag})
        item.update({'tags': dash.tags})
    
    if logging:
        
        adds = {"attributes":
            {
                "source_id": dash.id,
                "destination_id": dest_dash[0].id,
                "title": dest_dash[0].title,
                "owner": dest_dash[0].owner,
                "transfer_date": str(datetime.now())
            }
        }
        
        records.edit_features(adds=[adds])
        
    if swizzle:
        # Swizzle the old and new IDs
        cloned_dash = dest_dash[0]
        dash_json = cloned_dash.get_data()
        dash_str = json.dumps(dash_json)
        
        # build url dict
        wm_urls_keys = []
        wm_urls_vals = []
        
        # Swizzle Item IDs
        for key, val in wm_items.items():
            dash_str = dash_str.replace(key, val)
        
        updated_data = json.loads(dash_str)

        cloned_dash.update(item_properties = {}, data = updated_data)
 
        # Swizzle Feature Service URLs
        for key, val in wm_items.items():
            origin_wm = origin.content.get(key)
            origin_wm = WebMap(origin_wm)
            
            dest_wm = destination.content.get(val)
            dest_wm = WebMap(dest_wm)
            
            for layer in origin_wm.layers:
                wm_urls_keys.append(layer["url"])
            for layer in dest_wm.layers:
                wm_urls_vals.append(layer["url"])
        
        fs_url_dict = dict(zip(wm_urls_keys, wm_urls_vals))
        
        for key, val in fs_url_dict.items():
            dash_str.replace(key, val)
        
        updated_data = json.loads(dash_str)
        cloned_dash.update(item_properties = {}, data = updated_data)

    print(f"Dashboard Transfer: clone of Dashboard {cloned_dash.title} to folder {fldr_name} complete.")

def sm_transfer(destination: GIS,  
                item: Item,
                records: Table = None, 
                logging : bool = False) -> None:
    """
    Adapted code sample from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    
    Transfer protocol for Story Maps and their web content items. Does not call copy_items() as protocol is different for this content.
    
    Arguments:
        destination (GIS): Destination GIS for the origin item.
        item (Item): Story Map item to transfer from the origin.
        records (Table): Hosted Table item to memoize transfers to. Must be passed if logging=True
        logging (bool): If True, enables catalog memoization, pushing a transfer record to a Hosted Table in AGOL.  
    """
    
    story_map = item
    
    orig_thumbnail = story_map.download_thumbnail()
    
    if story_map.id == ITEM_ID:
        destination.content.create_folder(fldr_name) ##Changed Folder  
    
    # check version to apply relevant protocol
    if _version <= [1, 8, 2]:
        resource = export_resources(item=story_map)
    else:
        resource = story_map.resources.export()

    # get story map item data from json to store related maps
    story_map_json = story_map.get_data(try_json=True)

    web_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('webmap')>-1])
    express_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('expressmap')>-1])


    webmap_mapper = {} # keys are origin IDs, values are destination IDs
    for wm in web_maps:
        webmap_to_copy = origin.content.get(wm)
        
        if webmap_to_copy == None:
            print(f"Webmap Item {wm.title} in Storymap not found in the org. Skipping...")
            continue
        else:    
            
            cloned_webmaps = destination.content.clone_items([webmap_to_copy])
            webmap_mapper[webmap_to_copy.id] = [i for i in cloned_webmaps if i.type == 'Web Map'][0].id

            # memoize tranfer to catalog
            if logging:
                
                adds = {"attributes":
                    {
                        "source_id": webmap_to_copy.id,
                        "destination_id": webmap_mapper[webmap_to_copy.id],
                        "title": webmap_to_copy.title,
                        "owner": webmap_to_copy.owner,
                        "type": webmap_to_copy.type,
                        "transfer_date": str(datetime.now())
                    }
                }
                
                records.edit_features(adds=[adds])
                
            for wm in cloned_webmaps:
                try:
                    wm.move(fldr_name) ##Changed Folder
                except Exception:
                    continue
                
    # remap the old itemid to the new one
    story_map_text = json.dumps(story_map_json)

    for key, val in webmap_mapper.items():
        story_map_text = story_map_text.replace(key, val)

    new_item = destination.content.add({'type' : story_map.type,
                             'tags' : story_map.tags,
                             'title' : story_map.title,
                             'description' : story_map.description,
                             'typeKeywords' : story_map.typeKeywords,
                             'extent' : story_map.extent,
                             'text' :story_map_text}
                            )
    
    # bring in the storymap resources exported to a zip archive earlier
    new_item.resources.add(resource, archive=True)

    # update the url
    new_item.update({'url': story_map.url.replace(story_map.id, new_item.id)})
    new_item.update(thumbnail=orig_thumbnail)
    
    # set draft resources for the item
    with tempfile.NamedTemporaryFile(mode="w", suffix=".json", dir=tempfile.gettempdir(),
                                     delete=False) as jsonfile:
        jsonfile.write(json.dumps(new_item.get_data()))
        new_item.resources.add(file=jsonfile.name)
        type_keywords = [tk for tk in new_item.typeKeywords if 'smdraftresourceid' not in tk]
        type_keywords.append(f'smdraftresourceid:{os.path.basename(jsonfile.name)}')
        new_item.update({'typeKeywords': type_keywords})
    
    # express maps resources
    if len(express_maps) > 0:
        with tempfile.TemporaryDirectory() as d:
            shutil.unpack_archive(filename=resource, extract_dir=d)
            for expmap in express_maps:
                express_draft = os.path.join(d, "draft_" + expmap)
                express_pub = os.path.join(d, "pub" + expmap)
                if os.path.isfile(express_pub):
                    shutil.copy(express_pub, express_draft)
                    new_item.resources.add(express_draft)
    
    
    try:
        new_item.move(fldr_name) ##Changed Folder
    except Exception:
        print(f"{new_item} could not be moved. Ensure folder name is correct.")
    
    if logging:
    
        adds = {"attributes":
            {
                "source_id": item.id,
                "destination_id": new_item.id,
                "title": new_item.title,
                "owner": new_item.owner,
                "type": new_item.type,
                "transfer_date": str(datetime.now())
            }
        }
        
        records.edit_features(adds=[adds])
    
    print(f"Transfer of Storymap {new_item.title} to {fldr_name} complete.")

def exb_transfer(destination: GIS,  
                 exb: Item,
                 records: Table = None, 
                 logging : bool = False) -> None:
    """
    Transfer protocol for Experience Builder items.
    
    Arguments:
        destination (GIS): AGOL where the Experience is going to. 
        exb (Item): Experience Builder Item of interest.
        records (Table): Table which memoizes the transfer of Experience items. 
        logging (bool): If True, enables the memoization of transfers. Requires records argument to be specified. 
    """
    
    if exb.owner != ORIGIN_TRANSFER_USER and MOVE_ITEMS:
        exb.reassign_to(ORIGIN_TRANSFER_USER)
        
    experience = WebExperience(exb)
    new_experience = experience.clone(target=destination, owner=destination.properties.user.username, folder=fldr_name)
    
    exb_json = new_experience.get_data()

    old_embed_urls = []

    for widget in exb_json['widgets']:
            destination_embed_url = f'<p>{DESTINATION_URL}/apps/{new_embeds[widget].lower()}s/{new_embeds[widget].itemid}</p>'
            if exb_json['widgets'][widget]['uri'] == 'widgets/common/embed/':
                exb_json['widgets'][widget]['config']['expression'] = destination_embed_url

    # need to remove  HTML tags so URLs can be used to grab item ids
    old_embed_urls = [url.replace('<p>', "%temp%").replace("</p>", "").replace("%temp%", "") for url in old_embed_urls]
    old_embed_list = [destination.content.get(url.rsplit('/', 1)[-1]) for url in old_embed_urls]
    new_embeds = destination.content.clone_items(items=old_embed_list, folder=fldr_name)

    for widget in exb_json['widgets']:
        destination_embed_url = f'<p>{DESTINATION_URL}/apps/{new_embeds[widget].lower()}s/{new_embeds[widget].itemid}</p>'
        if exb_json['widgets'][widget]['uri'] == 'widgets/common/embed/':
            exb_json['widgets'][widget]['config']['expression'] = destination_embed_url
    
    new_experience.update(item_properties = {}, data = exb_json)
    
    if logging:
        
        adds = {"attributes":
            {
                "source_id": exb.id,
                "destination_id": new_experience.id,
                "title": new_experience.title,
                "owner": new_experience.owner,
                "type": new_experience.type,
                "transfer_date": str(datetime.now())
            }
        }
        
        records.edit_features(adds=[adds])
    
    print(f"Transfer of {new_experience.title} to folder {fldr_name} complete.")


# CLONING STARTS HERE (RERUN FROM HERE)
#### Run this section each time you change the item id

In [ ]:
catalog = destination.content.get(CATALOG_ID)
catalog = catalog.tables[0]

destination.content.get(CATALOG_ID)

In [ ]:
item_origin = origin.content.get(ITEM_ID)

print(f"You are attempting to clone {item_origin.title}: ")
item_origin

In [ ]:
# reassign item to origin transfer user
if item_origin.owner != ORIGIN_TRANSFER_USER and MOVE_ITEMS:
    item_origin.reassign_to(ORIGIN_TRANSFER_USER)

# decide on appropriate workflow for item
if item_origin.type == "StoryMap":
    destination.content.clone_items(items=[item_origin], folder=fldr_name)
elif item_origin.type == "Dashboard":
    dash_transfer(destination=destination, dash=item_origin, swizzle=True, records=catalog, logging=True)
elif item_origin.type == "Web Experience":
    exb_transfer(destination=destination, exb=item_origin, records=catalog, logging=True)
else:
    wc_transfer(destination=destination, items=[item_origin], records=catalog, logging=True)

print(f"Transfer of {item_origin.title} to folder {fldr_name} and its participating content complete. Please refresh your content page.")

#Move content to the final user that owns this and clean up the leftovers.
if REASSIGN == True and MOVE_TO != destination.properties.user.username:
    final_user = destination.users.get(MOVE_TO)
    finl_fldr_name = get_or_create_folder(destination, item_origin.title, final_user.username)

    mig_items = destination.users.get(destination.properties.user.username).items(folder=fldr_name)
    
    for item in mig_items:
        #print(item.title)
        item.reassign_to(final_user, target_folder=finl_fldr_name)
        print(f'{item.title} was moved from {destination.properties.user.username} to {MOVE_TO}.')

    # Check if the folder has any items
    if len(destination.users.me.items(folder=finl_fldr_name)) == 0:
        destination.content.delete_folder(finl_fldr_name, owner=destination.properties.user.username)
        print(f'Folder {finl_fldr_name} has no content in it and was deleted.')
    else:
        print(f'Folder {finl_fldr_name} STILL has content and was NOT deleted.')
        print(f'Script complete')
else:
    print(f'Script complete')